In [2]:
import numpy as np

In [3]:
def elimination(A, b):
    M = np.concatenate((A, b), axis=1)

    for k in range(0, M.shape[0]-1):
        for i in range(k+1, M.shape[0]):
            M[i] = M[i] - M[k] * M[i, k] / M[k, k]

    return np.split(M, [M.shape[1]-1], axis=1)

def STS(A, b):
    n = len(b)
    x = np.zeros(n)    
    for i in range(n-1, -1, -1):
        rest = sum([A[i, j] * x[j] for j in range(i+1, n)])
        x[i] = (b[i,0] - rest) / A[i,i]
    
    return x

In [6]:
A = np.array([2., -3., 1., 1., 2., 1., 4., -6., -1.]).reshape(3,3)
b = np.array([-5., 1., -7.]).reshape(3,1)
Anew, bnew = elimination(A, b)
print(STS(Anew, bnew))

[-0.28571429  1.14285714 -1.        ]
